<a href="https://colab.research.google.com/github/the-faisalahmed/Optimization/blob/main/Smart_Investment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://dmcommunity.org/wp-content/uploads/2024/06/image-3.png)

[Link](https://dmcommunity.org/challenge-july-2024/) to the original problem.

A client of an investment firm has $\$10000$ available for investment. He has instructed that his money be invested in particular stocks, so that no more than $\$5000$ is invested in any one stock but at least $\$1000$ be invested in each stock. He has further instructed the firm to use its current data and invest in the manner that maximizes his overall gain during a one-year period. The stocks, the current price per share and the firm’s predicted stock price a year from now are summarized below:


| Stock      | Current Price | Projected Price |
| ----------- | ----------- | ----------- |
| ABC      | \$25       | \$35       |
| XYZ      | \$50       | \$60       |
| TTT      | \$100       | \$125       |
| LMN      | \$25       | \$40       |

Your task is to create a decision model that can be used to make a smart investment while satisfying the client requirements for different combinations of stocks.


In [1]:
!pip install ortools
from ortools.sat.python import cp_model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.2 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobu

In [3]:
# Creates the model and set solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()

funds = 10000

max_alloc = 5000
min_alloc = 1000

stocks = {'ABC':{'Current': 25, 'Projected': 35},
          'XYZ': {'Current': 50, 'Projected': 60},
          'TTT': {'Current': 100, 'Projected': 125},
          'LMN': {'Current': 25, 'Projected': 40}}

x = {i:model.NewIntVar(0, max_alloc, f"x_{i}") for i in stocks.keys()}
num_stocks = {i:model.NewIntVar(0, max_alloc, f"num_stocks_{i}") for i in stocks.keys()}

# Must invest all available funds

model.Add(sum(x[i] for i in x.keys()) == funds)

for i in x.keys():
  # Must not invest more than max_alloc per investment
  model.Add(x[i] <= max_alloc)
  model.Add(x[i] >= min_alloc)
  model.AddDivisionEquality(num_stocks[i],x[i],stocks[i]['Current'])

# Maximize future gains

expression = [num_stocks[i]*(stocks[i]['Projected']) - x[i] for i in x.keys()]
model.Maximize(sum(expression))

status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")

Status = OPTIMAL


In [4]:
# Solution
for i in x.keys():
  print(f'Spend ${solver.value(x[i])} on stock {i}.')

print('\n')
print(f'Total gain = ${sum(solver.value(num_stocks[i])*(stocks[i]["Projected"]) - solver.value(x[i]) for i in x.keys())}')

Spend $3000 on stock ABC.
Spend $1000 on stock XYZ.
Spend $1000 on stock TTT.
Spend $5000 on stock LMN.


Total gain = $4650
